In [1]:
## Full backend analysis for the CREEDS and L1000 dataset 
import os
import pandas as pd
from pandas.compat import StringIO
import numpy as np
from numpy import loadtxt
import sys
import json
from pprint import pprint
import objectpath
import csv
import re
import matplotlib.pyplot as plt
import json, requests
from pprint import pprint
import itertools
import scipy
from scipy.spatial import distance
from sklearn.metrics.pairwise import pairwise_distances
from clustergrammer_widget import *
def get_geneset(df, indexer):
    df_ = df.loc[indexer, :]
    return list(df_[df_ == 1].index)

In [2]:
## load in the pre-formed datasets from the L1000_Analysis and CREEDS_Analysis files.
## THIS TAKES A WHILE TO LOAD, SO ONLY LOAD THIS ONCE AND EARLY

# L1000 up and down gene loads for drug signatures
#L1000_up_genes = pd.read_csv("L1000_up_genes.csv")
#L1000_down_genes = pd.read_csv("L1000_down_genes.csv")

# CREEDS up and down genes for disease signatures
with open("disease_signatures-v1.0.json") as f:
    CREEDS_data = json.load(f)

# generate the up and down gene signatures
#CREEDS_up_genes = {
#    row['do_id']: row['up_genes']
#    for row in CREEDS_data
#}
#CREEDS_down_genes = {
#    row['do_id']: row['down_genes']
#    for row in CREEDS_data
#}

# RETURNS THE do_id, geo_id, and disease name in a dictionary
CREEDS_GSE = {
    row['id']: [row['geo_id'], row["disease_name"]]
    for row in CREEDS_data
}

# load in the EMR Data (filtered > 200 in R code [Drug_diagnosis_test_code.R])
EMR_data = pd.read_csv("EMR_greater_200.csv")
## subset EMR data by the DOI and/or DrOI
EMR_data_df = pd.DataFrame(EMR_data)
#EMR_data
EMR_data_df.drop(EMR_data_df.columns[[0]], axis = 1, inplace = True) # remove the unecessary columns
#EMR_data_df

# implement the search from ICD9-do_id from the manual conversion
icd9_to_doid = pd.read_csv("ICD9_CREEDS_conversion.csv")
icd9_to_doid = pd.DataFrame(icd9_to_doid) # convert it to a data fram to drop unecessary rows
#icd9_to_doid # sanity check
icd9_to_doid_final = icd9_to_doid.drop(icd9_to_doid.columns[[0, 6, 7, 8, 9, 10, 11, 12, 13, 14]], axis = 1)
#icd9_to_doid_final # sanity check


posssible_disease_list = set(icd9_to_doid_final.Disease) 
#& set(EMR_data_df.Description) # will return {"Alzheimer's disease", "Barrett's esophagus", 'Dehydration', 'Sepsis'}

posssible_disease_list = list(posssible_disease_list) # this will be what the dropdown should display

In [3]:
len(posssible_disease_list)

327

In [27]:
len(CREEDS_GSE.keys())

828

In [28]:
### load in the CREEDS and the L1000 disease and drug signatures respectively 

with open ("L1000_total_up_genes_drug_sigs.json", 'r') as f:
    L1000_total_up_genes_drug_sigs = json.load(f)

with open ("L1000_total_down_genes_drug_sigs.json", 'r') as f:
    L1000_total_down_genes_drug_sigs = json.load(f)

with open ("CREEDS_total_up_disease_id_signatures_total_no_scores.json", 'r') as f:
    CREEDS_total_up_disease_id_signatures_total_no_scores = json.load(f)

for key in CREEDS_total_up_disease_id_signatures_total_no_scores:
    #if type(my_dict[key]) == type([]):
    for index, item in enumerate(CREEDS_total_up_disease_id_signatures_total_no_scores[key]):
        CREEDS_total_up_disease_id_signatures_total_no_scores[key][index] = item.upper();
    
    
with open ("CREEDS_total_down_disease_id_signatures_total_no_scores.json", 'r') as f:
    CREEDS_total_down_disease_id_signatures_total_no_scores = json.load(f)

for key in CREEDS_total_down_disease_id_signatures_total_no_scores:
    #if type(my_dict[key]) == type([]):
    for index, item in enumerate(CREEDS_total_down_disease_id_signatures_total_no_scores[key]):
        CREEDS_total_down_disease_id_signatures_total_no_scores[key][index] = item.upper();
    
    
with open("L1000_CREEDS_Similar_Drug_output.txt", "r") as fd:
    similar_L1000_CREEDS_drugs = fd.read().splitlines()
    
with open ("synonyms.json", 'r') as f:
    synlist = json.load(f)

In [30]:
len(L1000_total_down_genes_drug_sigs.keys())

42809

In [19]:
len(posssible_disease_list)

327

In [20]:
while("" in posssible_disease_list) : 
    posssible_disease_list.remove("") 

In [22]:
posssible_disease_list

['colon adenoma',
 'relapsing-remitting multiple sclerosis',
 'Helicobacter pylori gastrointestinal tract infection',
 'autistic disorder',
 'Epithelial proliferation',
 'Rotavirus infection children',
 'Generalized seizures',
 'focal facial dermal dysplasia type III',
 'vitamin A deficiency',
 'Hypertrophy',
 'Familial hypophosphataemic rickets',
 'dermatomyositis',
 'obstructive sleep apnea',
 'Duchenne muscular dystrophy',
 'Gamma-hydroxybutyric acidaemia',
 'nephrolithiasis',
 'esophagus adenocarcinoma',
 'papillary thyroid carcinoma',
 "Sjogren's syndrome",
 'idiopathic interstitial pneumonia',
 'dilated cardiomyopathy',
 'adenoma',
 'Waldenstrom macroglobulinemia',
 'liver cirrhosis',
 'Wolfram syndrome',
 'bone fracture',
 'myotonic dystrophy type 1',
 'hypertension',
 'acute T cell leukemia',
 'meningococcal infection',
 'Dental cavity',
 'lung squamous cell carcinoma',
 'appendicitis',
 'cellular phase chronic idiopathic myelofibrosis',
 'Rhinovirus infection',
 "Parkinson's d

In [17]:
###### GENERATE THE FULL ANALYSIS BETWEEN CREEDS AND L1000 GIVEN A DISEASE OF INTEREST

# USER INPUT
####
DOI = "asthma" # disease of interest. CAN TAKE FROM posssible_disease_list FOR NOW
####
#possible_diseases = EMR_data_df["Description"] #possible diseases from the Sinai EMR

### DISEASE --> ICD9 --> DOI
# get the ICD9 from the DOI
DOI_ICD9 = icd9_to_doid_final[icd9_to_doid_final.Disease.apply(lambda s: bool(re.compile(DOI, re.IGNORECASE).search(s)))]
DOI_ICD9_codes = DOI_ICD9.ICD9

# get the do_id from the DOI
DOI_DOID_codes = DOI_ICD9.DOID

# get the do_id from the DOI
DOI_CREEDS_codes = DOI_ICD9.CREEDS_drug_id

# INPUT SIMILAR CREEDS CODE FROFM THE DRUG_QUERY

#DOIDs_up = {
#    doid: geneset
#    for doid, geneset in CREEDS_up_genes.items()
#    if doid in set(DOI_DOID_codes)
#}

#DOIDs_down = {
#    doid: geneset
#    for doid, geneset in CREEDS_down_genes.items()
#    if doid in set(DOI_DOID_codes)
#}
    
##filter by DOI. Need to convert DOI to ICD9 first.
icd9_to_doid_final_search = icd9_to_doid_final[icd9_to_doid_final["Disease"].apply(lambda s: bool(re.compile(DOI, re.IGNORECASE).search(s)))]
icd9_to_doid_final_search1 = icd9_to_doid_final_search["ICD9"]

## rebuild the wildcard dataframe
icd9_wildcard = pd.DataFrame(icd9_to_doid_final_search1, columns=['ICD9'])
icd9_wildcard['ICD9_wildcard'] = icd9_wildcard['ICD9'].apply(lambda code: str(code).split('.')[0])
icd9_wildcard.head()

icd9_to_doid_final['ICD9_wildcard'] = icd9_to_doid_final['ICD9'].apply(lambda code: str(code).split('.')[0])
#icd9_to_doid_final.head()
ICD9_df_joined = pd.merge(
    left=icd9_wildcard, left_on='ICD9_wildcard',
    right=icd9_to_doid_final, right_on='ICD9_wildcard',
    how='inner',
    suffixes=(
        '_Manual',
        '_right',
    )
)


#ICD9_codes = str(int(ICD9_df_joined["ICD9_wildcard"].unique())) 
## generate an emr based on the ICD_9 codes extracted; can now extract the drug names as well
#emr_sub = EMR_data_df[EMR_data_df['ICD9'].apply(lambda s: bool(re.compile(ICD9_codes, re.IGNORECASE).search(s)))]
emr_sub = EMR_data_df[EMR_data_df["Description"].apply(lambda s: bool(re.compile(str(DOI), re.IGNORECASE).search(str(s))))]
#emr_sub[0:10]
emr_sub.reset_index(drop = True, inplace = True)

emr_sub = []
for a in ICD9_df_joined.ICD9_wildcard.unique():
    emr_sub1 = EMR_data_df[EMR_data_df["ICD9"].apply(lambda s: bool(re.compile(str(a), re.IGNORECASE).search(str(s))))]
    emr_sub.append(emr_sub1)
emr_sub_df = pd.concat(emr_sub)
#### L1000 integration
# disease to drug conversion (disease input)
top_drugs_from_disease = list(emr_sub_df.Drug_Name[0:10]) #take the top 5 drugs
test = pd.DataFrame(L1000_down_genes)
#for x in top_drugs_from_disease:
#    print(x)
#    top_drug_disease_extract = test[test['Unnamed: 0'].apply(lambda s: bool(re.compile(str(x), re.IGNORECASE).search(str(s))))]
metadata = pd.read_csv("L1000_metadata.csv")
metadata ## same as LINC1000h5.row_metadata_df
#metadata
for a in top_drugs_from_disease:
    print(a)
    meta_doi = metadata[metadata["pert_desc"].apply(lambda s: bool(re.compile(str(a), re.IGNORECASE).search(str(s))))]
meta_doi
meta_doi_ids = meta_doi.rid
query = list(meta_doi_ids)
#print(query)
# disease to drug conversion (disease input)
top_drugs_from_disease = list(emr_sub_df.Drug_Name[0:20]) #take the top 5 drugs
test = pd.DataFrame(L1000_down_genes)
top_drugs_from_disease


single_word_drugs = []
for i in top_drugs_from_disease:
    j = str(i)
    splitted = j.split()
    first_word = splitted[0]
   # print(first_word)
    single_word_drugs.append(first_word) 
#print(single_word_drugs)
single_word_drugs = list(set(single_word_drugs))
#single_word_drugs

# Generate a blacklist process
def process_blacklist(s):
    import re
    blacklist = [
        # remove the classifications of the drugs
        re.compile(r'INJ', re.IGNORECASE),
        re.compile(r'CAP', re.IGNORECASE),
        re.compile(r'\d+', re.IGNORECASE),
        
        # remove drugs that aren't in the L1000
        re.compile(r'SODIUM', re.IGNORECASE),
        re.compile(r'HEPATITIS', re.IGNORECASE),
        re.compile(r'HEPARIN', re.IGNORECASE),
        re.compile(r'CALCIUM', re.IGNORECASE),
        
    ]
    for b in blacklist:
        s = re.sub(b, '', s)
    return s.strip()
single_word_drugs_list = list(pd.Series(single_word_drugs).map(process_blacklist))
single_word_drugs_list


## L1000 API Integration
L1000FWD_URL = 'http://amp.pharm.mssm.edu/L1000FWD/'


L1000_reverse_drugs_store = []


for a in single_word_drugs_list:
    query_string = a
    L1000_reverse_drug_response = requests.get(L1000FWD_URL + 'synonyms/' + query_string)
    if L1000_reverse_drug_response.status_code == 200:
        #pprint(L1000_reverse_drug_response.json())
        L1000_reverse_significant_query = L1000_reverse_drug_response.json()
        if len(L1000_reverse_significant_query) > 0:
            json.dump(L1000_reverse_drug_response.json(), open(query_string + '_L1000_reverse_drug_query.json', 'w'), indent=4)
            L1000_reverse_significant_query = L1000_reverse_drug_response.json()
            L1000_reverse_significant_query_df = pd.DataFrame(L1000_reverse_significant_query)
            L1000_reverse_drugs_store.append(a)
            print("Found significant L1000 drug signatures for " + query_string)
        else:
            print("No significant L1000 drug signatures for " + query_string)
  
    
    


ALBUTEROL SULFATE HFA 90 MCG/ACTUATION AEROSOL INHALER
ALBUTEROL SULFATE 2.5 MG/3 ML (0.083 %) NEB SOLUTION
HEPARIN SODIUM INJ
DOCUSATE SODIUM 100 MG CAPSULE
ALBUTEROL SULFATE 2.5 MG/3 ML (0.083 %) NEB SOLUTION
DOCUSATE SODIUM CAP
HEPARIN (PORCINE) 5,000 UNIT/ML INJECTION
SODIUM CHLORIDE 0.9%
ZZ IMS TEMPLATE
ALBUTEROL 90 MCG/ACTUATION AEROSOL INHALER
Found significant L1000 drug signatures for ALBUTEROL
Found significant L1000 drug signatures for DOCUSATE
Found significant L1000 drug signatures for ESOMEPRAZOLE
No significant L1000 drug signatures for ZZ
No significant L1000 drug signatures for Fentanyl
Found significant L1000 drug signatures for PREDNISONE
Found significant L1000 drug signatures for ASPIRIN
Found significant L1000 drug signatures for IPRATROPIUM
No significant L1000 drug signatures for OXYCODONE-ACETAMINOPHEN
Found significant L1000 drug signatures for Propofol
Found significant L1000 drug signatures for MONTELUKAST


In [4]:
top_drugs_from_disease

['ALBUTEROL SULFATE HFA 90 MCG/ACTUATION AEROSOL INHALER',
 'ALBUTEROL SULFATE 2.5 MG/3 ML (0.083 %) NEB SOLUTION',
 'HEPARIN SODIUM INJ',
 'DOCUSATE SODIUM 100 MG CAPSULE',
 'ALBUTEROL SULFATE 2.5 MG/3 ML (0.083 %) NEB SOLUTION',
 'DOCUSATE SODIUM CAP',
 'HEPARIN (PORCINE) 5,000 UNIT/ML INJECTION',
 'SODIUM CHLORIDE 0.9%',
 'ZZ IMS TEMPLATE',
 'ALBUTEROL 90 MCG/ACTUATION AEROSOL INHALER',
 'Propofol',
 'OXYCODONE-ACETAMINOPHEN 5 MG-325 MG TABLET',
 'Fentanyl',
 'IPRATROPIUM BROMIDE 0.02 % SOLN FOR INHALATION',
 'IPRATROPIUM BROMIDE 0.02 % SOLN FOR INHALATION',
 'MONTELUKAST 10 MG TABLET',
 'ALBUTEROL SULFATE 2.5 MG/3 ML (0.083 %) SOLUTION FOR NEBULIZATION',
 'PREDNISONE 20 MG TABLET',
 'ASPIRIN 81 MG CHEWABLE TABLET',
 'ESOMEPRAZOLE CAP']

In [7]:
single_word_drugs_list

['ALBUTEROL',
 'DOCUSATE',
 'ESOMEPRAZOLE',
 'ZZ',
 'Fentanyl',
 'PREDNISONE',
 '',
 'ASPIRIN',
 'IPRATROPIUM',
 'OXYCODONE-ACETAMINOPHEN',
 'Propofol',
 'MONTELUKAST']

['ALBUTEROL',
 'DOCUSATE',
 'ESOMEPRAZOLE',
 'ZZ',
 'Fentanyl',
 'PREDNISONE',
 'ASPIRIN',
 'IPRATROPIUM',
 'OXYCODONE-ACETAMINOPHEN',
 'Propofol',
 'MONTELUKAST']

In [14]:
### CREEDS DRUG CARD 
while("" in single_word_drugs_list):
    single_word_drugs_list.remove("")
    
single_word_drugs_list
CREEDS_URL = 'http://amp.pharm.mssm.edu/CREEDS/'

CREEDS_drug_from_disease_up_genes = []
CREEDS_drug_from_disease_down_genes = []

for a in single_word_drugs_list:
    CREEEDS_Drug_response = requests.get(CREEDS_URL + 'search', params={'q':str(a)})
    if CREEEDS_Drug_response.status_code == 200:
        #pprint(CREEEDS_Drug_response.json())
        #json.dump(CREEEDS_Drug_response.json(), open(DrOI + '_api1_result.json', 'w'), indent=4)
        CREEDS_drug_output_df = pd.DataFrame(CREEEDS_Drug_response.json())
        
        if len(CREEDS_drug_output_df) > 0:
            CREEDS_drug_output_ids = list(CREEDS_drug_output_df["id"])
            print("CREEDS IDs found for " + a)
            for a in CREEDS_drug_output_ids:
                CREEDS_drug_sigs_response = requests.get(CREEDS_URL + 'api', params={'id':'drug:DM609'})
                if CREEDS_drug_sigs_response.status_code == 200:
                    CREEDS_drug_sigs_response_json = CREEDS_drug_sigs_response.json()

                    ## up genes
                    CREEDS_drug_sigs_up_genes = CREEDS_drug_sigs_response_json['up_genes']
                    CREEDS_drug_sigs_up_genes_df = pd.DataFrame(CREEDS_drug_sigs_up_genes) # this is the up genes dataframe
                    filename1 = (a + "_CREEDS_drug_sig_up_genes.csv")
                    #CREEDS_drug_sigs_up_genes_df.to_csv(filename1) # this saves the df as a csv

                    ## down genes
                    CREEDS_drug_sigs_down_genes = CREEDS_drug_sigs_response_json['down_genes']
                    CREEDS_drug_sigs_down_genes_df = pd.DataFrame(CREEDS_drug_sigs_down_genes)# this is the down genes dataframe
                    filename2 = (a + "_CREEDS_drug_sig_down_genes.csv")
                    print(filename2)
                    #CREEDS_drug_sigs_down_genes_df.to_csv(filename2)
                    #CREEDS_drug_sigs_down_genes = CREEDS_drug_sigs_response_json['down_genes'] # this saves the df as a csv

                    ## json propagation
                    #pprint(response.json())
                    #json.dump(response.json(), open(a + '_CREEDS_Drug_sig.json', 'w'), indent=4) # if the user wants the entire json, they can download this

                    CREEDS_drug_from_disease_up_genes.append(CREEDS_drug_sigs_up_genes)
                    CREEDS_drug_from_disease_down_genes.append(CREEDS_drug_sigs_down_genes_df)
        else:
            print ("No CREEDS IDs found for " + a)


No CREEDS IDs found for ALBUTEROL
No CREEDS IDs found for DOCUSATE
No CREEDS IDs found for ESOMEPRAZOLE
CREEDS IDs found for ZZ
gene:P4427_CREEDS_drug_sig_down_genes.csv
gene:P9118_CREEDS_drug_sig_down_genes.csv
gene:P9119_CREEDS_drug_sig_down_genes.csv
gene:P9120_CREEDS_drug_sig_down_genes.csv
gene:P9121_CREEDS_drug_sig_down_genes.csv
gene:P9122_CREEDS_drug_sig_down_genes.csv
gene:P9123_CREEDS_drug_sig_down_genes.csv
gene:P9279_CREEDS_drug_sig_down_genes.csv
No CREEDS IDs found for Fentanyl
CREEDS IDs found for PREDNISONE
drug:DM516_CREEDS_drug_sig_down_genes.csv
drug:DM517_CREEDS_drug_sig_down_genes.csv
drug:DM518_CREEDS_drug_sig_down_genes.csv
drug:DM1183_CREEDS_drug_sig_down_genes.csv
drug:DM1184_CREEDS_drug_sig_down_genes.csv
drug:DM1185_CREEDS_drug_sig_down_genes.csv
CREEDS IDs found for ASPIRIN
drug:DM568_CREEDS_drug_sig_down_genes.csv
drug:DM569_CREEDS_drug_sig_down_genes.csv
drug:DM1286_CREEDS_drug_sig_down_genes.csv
drug:DM1287_CREEDS_drug_sig_down_genes.csv
drug:DM1288_CREED

In [11]:
CREEDS_drug_output_df

""


In [ ]:
### CREEDS DRUG CARD 
single_word_drugs_list
CREEDS_URL = 'http://amp.pharm.mssm.edu/CREEDS/'

CREEDS_drug_from_disease_up_genes = []
CREEDS_drug_from_disease_down_genes = []

for a in single_word_drugs_list:
    CREEEDS_Drug_response = requests.get(CREEDS_URL + 'search', params={'q':str(a)})
    if CREEEDS_Drug_response.status_code == 200:
        #pprint(CREEEDS_Drug_response.json())
        #json.dump(CREEEDS_Drug_response.json(), open(DrOI + '_api1_result.json', 'w'), indent=4)
        CREEDS_drug_output_df = pd.DataFrame(CREEEDS_Drug_response.json())
        CREEDS_drug_output_ids = list(CREEDS_drug_output_df["id"])

        for a in CREEDS_drug_output_ids:
            CREEDS_URL = 'http://amp.pharm.mssm.edu/CREEDS/'
            CREEDS_drug_sigs_response = requests.get(CREEDS_URL + 'api', params={'id':'drug:DM609'})
            if CREEDS_drug_sigs_response.status_code == 200:
                CREEDS_drug_sigs_response_json = CREEDS_drug_sigs_response.json()

                ## up genes
                CREEDS_drug_sigs_up_genes = CREEDS_drug_sigs_response_json['up_genes']
                CREEDS_drug_sigs_up_genes_df = pd.DataFrame(CREEDS_drug_sigs_up_genes) # this is the up genes dataframe
                filename1 = (a + "_CREEDS_drug_sig_up_genes.csv")
                #CREEDS_drug_sigs_up_genes_df.to_csv(filename1) # this saves the df as a csv

                ## down genes
                CREEDS_drug_sigs_down_genes = CREEDS_drug_sigs_response_json['down_genes']
                CREEDS_drug_sigs_down_genes_df = pd.DataFrame(CREEDS_drug_sigs_down_genes)# this is the down genes dataframe
                filename2 = (a + "_CREEDS_drug_sig_down_genes.csv")
                #CREEDS_drug_sigs_down_genes_df.to_csv(filename2)
                #CREEDS_drug_sigs_down_genes = CREEDS_drug_sigs_response_json['down_genes'] # this saves the df as a csv

                ## json propagation
                #pprint(response.json())
                #json.dump(response.json(), open(a + '_CREEDS_Drug_sig.json', 'w'), indent=4) # if the user wants the entire json, they can download this
                
                CREEDS_drug_from_disease_up_genes.append(CREEDS_drug_sigs_up_genes)
                CREEDS_drug_from_disease_down_genes.append(CREEDS_drug_sigs_down_genes_df)

In [26]:
L1000_reverse_drugs_store

['IPRATROPIUM',
 'ASPIRIN',
 'ESOMEPRAZOLE',
 'ALBUTEROL',
 'DOCUSATE',
 'MONTELUKAST',
 'Propofol',
 'PREDNISONE']

In [44]:
## query L1000 from drug name
L1000FWD_URL = 'http://amp.pharm.mssm.edu/L1000FWD/'
#L1000_disease_query_store = []
L1000_disease_pert_id_store = []
L1000_disease_signatures_store = []
for a in L1000_reverse_drugs_store:
    query_string = a
    response = requests.get(L1000FWD_URL + 'synonyms/' + query_string)
    if response.status_code == 200:
        #pprint(response.json())
        #json.dump(response.json(), open('api1_result.json', 'wb'), indent=4)
        L1000_significant_query = response.json()
        L1000_significant_query_df = pd.DataFrame(L1000_significant_query)
        L1000_significant_pert_ids = L1000_significant_query_df["pert_id"]
        L1000_disease_pert_id_store.append(list(L1000_significant_pert_ids))
        L1000_summary_table = metadata[metadata["pert_id"].isin(L1000_significant_pert_ids)] # display this table!
        L1000_drug_signatures = L1000_summary_table.rid
        L1000_disease_signatures_store.append(list(L1000_drug_signatures))
        
        L1000_disease_pert_id_store = list(itertools.chain.from_iterable(L1000_disease_pert_id_store))
        
        if len(L1000_disease_pert_id_store) > 0:

            test=[]
            for q in L1000_disease_pert_id_store:
                meta_doi = metadata[metadata["pert_id"].apply(lambda s: bool(re.compile(str(q), re.IGNORECASE).search(str(s))))]
                #print(q)
                meta_doi_ids = meta_doi.rid
                query = list(meta_doi_ids)
               # print(query)
                test.append(query)
            #test
            test1 = [x for x in test if x]
           #test1
            L1000_disease_signatures = list(itertools.chain.from_iterable(test1))


            if len(L1000_disease_signatures) > 0:
            ## L1000 iterations API   
                L1000FWD_URL = 'http://amp.pharm.mssm.edu/L1000FWD/'
                json_name_store = []
                significant_signatures_names = []
               # disease_sig_store = []
                for x in L1000_disease_signatures:
                    sig_id = x
                    response = requests.get(L1000FWD_URL + 'sig/' + str(sig_id))
                    if response.status_code == 200:
                        #pprint(response.json())
                        json.dump(response.json(), open(sig_id + '_api2_result.json', 'w'), indent=4) #this is the saved json
                        json_name = (sig_id + '_api2_result.json')
                        print(json_name)
                        json_name_store.append(json_name) # these are all the names stores in a list that you can use to display (has.json after it)
                        significant_signatures_names.append(sig_id)
                        L1000_disease_query = response.json() 
                        L1000_disease_query_up_genes = L1000_disease_query["up_genes"]
                        L1000_disease_query_down_genes = L1000_disease_query["down_genes"]


                #L1000_up_df = pd.DataFrame(L1000_query_up_genes)
                #L1000_down_df = pd.DataFrame(L1000_query_down_genes)
                #L1000_down_df = L1000_down_df.T
                #L1000_up_df = L1000_up_df.T
                #L1000_up_df

                print("SIGNIFICANT L1000 SIGNATURES")
            else:
                print ("NO Significant signatures found in metadata")


        else:
            print("NO SIGNIFICANT L1000 SIGNATURES FOR" + DrOI)


CPC004_HEPG2_6H:BRD-A05352148-004-02-0:10_api2_result.json
CPC020_HA1E_6H:BRD-A05352148-001-02-6:10_api2_result.json
CPC001_VCAP_24H:BRD-A07029265-004-02-9:10_api2_result.json
CPC017_HEPG2_6H:BRD-A07029265-004-02-9:10_api2_result.json
CPC017_MCF7_24H:BRD-A07029265-004-02-9:10_api2_result.json
CPC017_NPC_24H:BRD-A07029265-004-02-9:10_api2_result.json
CPC005_A549_6H:BRD-A37432947-004-02-5:10_api2_result.json
CPC005_HT29_6H:BRD-A37432947-004-02-5:10_api2_result.json
CPC005_VCAP_24H:BRD-A37432947-004-02-5:10_api2_result.json
CPC005_VCAP_6H:BRD-A37432947-004-02-5:10_api2_result.json
SIGNIFICANT L1000 SIGNATURES
AML001_CD34_6H:BRD-K43389675:10_api2_result.json
AML001_PC3_6H:BRD-A19037878:0.37037_api2_result.json
AML001_PC3_6H:BRD-A19037878:1.11111_api2_result.json
AML001_PC3_6H:BRD-A19037878:10_api2_result.json
AML001_PC3_6H:BRD-A19037878:3.33333_api2_result.json
AML001_PC3_6H:BRD-A19500257:0.37037_api2_result.json
AML001_PC3_6H:BRD-A19500257:1.11111_api2_result.json
AML001_PC3_6H:BRD-A19500

ASG001_MCF7_6H:BRD-A90490067-001-02-7:2_api2_result.json
ASG001_MCF7_6H:BRD-K18910433-001-14-3:0.08_api2_result.json
ASG001_MCF7_6H:BRD-K18910433-001-14-3:0.4_api2_result.json
ASG001_MCF7_6H:BRD-K18910433-001-14-3:10_api2_result.json
ASG001_MCF7_6H:BRD-K18910433-001-14-3:2_api2_result.json
ASG001_MCF7_6H:BRD-K21680192-300-08-6:0.08_api2_result.json
ASG001_MCF7_6H:BRD-K21680192-300-08-6:0.4_api2_result.json
ASG001_MCF7_6H:BRD-K21680192-300-08-6:10_api2_result.json
ASG001_MCF7_6H:BRD-K21680192-300-08-6:2_api2_result.json
ASG001_MCF7_6H:BRD-K27305650-001-06-7:0.08_api2_result.json
ASG001_MCF7_6H:BRD-K27305650-001-06-7:0.4_api2_result.json
ASG001_MCF7_6H:BRD-K27305650-001-06-7:10_api2_result.json
ASG001_MCF7_6H:BRD-K27305650-001-06-7:2_api2_result.json
ASG001_MCF7_6H:BRD-K41260949-001-06-7:0.08_api2_result.json
ASG001_MCF7_6H:BRD-K41260949-001-06-7:0.4_api2_result.json
ASG001_MCF7_6H:BRD-K41260949-001-06-7:10_api2_result.json
ASG001_MCF7_6H:BRD-K41260949-001-06-7:2_api2_result.json
ASG001_

BRAF001_A375_24H:BRD-K38436528-003-20-4:10_api2_result.json
BRAF001_A375_24H:BRD-K53737926-003-18-7:0.625_api2_result.json
BRAF001_A375_24H:BRD-K63675182-003-18-6:10_api2_result.json
BRAF001_A375_24H:BRD-K67578145-001-05-5:0.15625_api2_result.json
BRAF001_A375_24H:BRD-K67578145-001-05-5:0.625_api2_result.json
BRAF001_A375_24H:BRD-K67578145-001-05-5:10_api2_result.json
BRAF001_A375_24H:BRD-K67578145-001-05-5:2.5_api2_result.json
BRAF001_A375_24H:BRD-K81418486-001-15-2:0.15625_api2_result.json
BRAF001_A375_24H:BRD-K81418486-001-15-2:0.625_api2_result.json
BRAF001_A375_24H:BRD-K81418486-001-15-2:10_api2_result.json
BRAF001_A375_24H:BRD-K81418486-001-15-2:2.5_api2_result.json
BRAF001_A375_24H:BRD-K92049597-001-14-1:10_api2_result.json
BRAF001_A375_6H:BRD-A19037878-001-01-5:0.15625_api2_result.json
BRAF001_A375_6H:BRD-A19037878-001-01-5:0.625_api2_result.json
BRAF001_A375_6H:BRD-A19037878-001-01-5:10_api2_result.json
BRAF001_A375_6H:BRD-A19037878-001-01-5:2.5_api2_result.json
BRAF001_A375_6

CPC001_HA1E_24H:BRD-K76674262-001-01-7:10_api2_result.json
CPC001_HA1E_24H:BRD-K77133231-001-01-1:10_api2_result.json
CPC001_HA1E_24H:BRD-K77998258-001-02-3:10_api2_result.json
CPC001_HA1E_24H:BRD-K78692225-001-11-2:10_api2_result.json
CPC001_HA1E_24H:BRD-K81209512-001-01-3:10_api2_result.json
CPC001_HA1E_24H:BRD-K81418486:10_api2_result.json
CPC001_HA1E_24H:BRD-K81709173-001-03-8:10_api2_result.json
CPC001_HA1E_24H:BRD-K82688027-001-01-5:10_api2_result.json
CPC001_HA1E_24H:BRD-K82823804-001-01-7:10_api2_result.json
CPC001_HA1E_24H:BRD-K84421793-001-01-1:10_api2_result.json
CPC001_HA1E_24H:BRD-K84595254-001-03-0:10_api2_result.json
CPC001_HA1E_24H:BRD-K84709232-001-02-6:10_api2_result.json
CPC001_HA1E_24H:BRD-K84987553-001-01-1:10_api2_result.json
CPC001_HA1E_24H:BRD-K85015012-003-01-1:10_api2_result.json
CPC001_HA1E_24H:BRD-K85242180-001-01-1:10_api2_result.json
CPC001_HA1E_24H:BRD-K86958018-001-01-9:10_api2_result.json
CPC001_HA1E_24H:BRD-K87510569-001-01-0:10_api2_result.json
CPC001

CPC001_HA1E_6H:BRD-K94832621-001-01-5:10_api2_result.json
CPC001_HA1E_6H:BRD-K95202259-001-01-9:10_api2_result.json
CPC001_HA1E_6H:BRD-K97061094-001-03-2:10_api2_result.json
CPC001_HA1E_6H:BRD-K97330509-001-01-4:10_api2_result.json
CPC001_HA1E_6H:BRD-K97365803-001-01-3:10_api2_result.json
CPC001_HCC515_24H:BRD-A01295252-001-01-5:10_api2_result.json
CPC001_HCC515_24H:BRD-A03816571-001-01-0:10_api2_result.json
CPC001_HCC515_24H:BRD-A06276885-003-01-1:10_api2_result.json
CPC001_HCC515_24H:BRD-A14574269-051-01-4:10_api2_result.json
CPC001_HCC515_24H:BRD-A15530910-001-01-3:10_api2_result.json
CPC001_HCC515_24H:BRD-A17065207-001-06-9:10_api2_result.json
CPC001_HCC515_24H:BRD-A19037878:10_api2_result.json
CPC001_HCC515_24H:BRD-A19374631-001-01-0:10_api2_result.json
CPC001_HCC515_24H:BRD-A19500257:10_api2_result.json
CPC001_HCC515_24H:BRD-A22783572-065-01-3:10_api2_result.json
CPC001_HCC515_24H:BRD-A23723433-001-01-2:10_api2_result.json
CPC001_HCC515_24H:BRD-A24429032-003-01-6:10_api2_result.j

...

CPC006_PC3_24H:BRD-A85860691-001-02-2:0.08_api2_result.json
CPC006_PC3_24H:BRD-A93255169-001-13-5:10_api2_result.json
CPC006_PC3_24H:BRD-A97437073-001-03-1:80_api2_result.json
CPC006_PC3_24H:BRD-K00615600-001-06-1:25_api2_result.json
CPC006_PC3_24H:BRD-K01121114-001-01-9:10_api2_result.json
CPC006_PC3_24H:BRD-K01292756-001-10-2:10_api2_result.json
CPC006_PC3_24H:BRD-K01779529-236-01-5:44.4_api2_result.json
CPC006_PC3_24H:BRD-K01868942-001-02-1:160_api2_result.json
CPC006_PC3_24H:BRD-K01877528-001-01-6:10_api2_result.json
CPC006_PC3_24H:BRD-K01976263-003-04-5:0.63_api2_result.json
CPC006_PC3_24H:BRD-K02113016-001-01-4:44.4_api2_result.json
CPC006_PC3_24H:BRD-K02407574-001-04-8:0.31_api2_result.json
CPC006_PC3_24H:BRD-K02526760-001-01-5:10_api2_result.json
CPC006_PC3_24H:BRD-K02822062-001-01-3:10_api2_result.json
CPC006_PC3_24H:BRD-K02950022-001-01-1:10_api2_result.json
CPC006_PC3_24H:BRD-K03618428-001-01-3:22.2_api2_result.json
CPC006_PC3_24H:BRD-K03816923-001-04-7:9.68_api2_result.json

CPC006_PC3_24H:BRD-K73789395-001-06-5:102.71_api2_result.json
CPC006_PC3_24H:BRD-K74148702-001-03-8:48_api2_result.json
CPC006_PC3_24H:BRD-K74305673-001-01-3:10_api2_result.json
CPC006_PC3_24H:BRD-K74402642-003-01-2:10_api2_result.json
CPC006_PC3_24H:BRD-K76205745-237-05-8:160_api2_result.json
CPC006_PC3_24H:BRD-K76703230-001-01-3:0.31_api2_result.json
CPC006_PC3_24H:BRD-K76964878-001-01-8:24_api2_result.json
CPC006_PC3_24H:BRD-K77947974-001-05-6:4.21_api2_result.json
CPC006_PC3_24H:BRD-K78126613-001-16-0:10_api2_result.json
CPC006_PC3_24H:BRD-K78513633-001-07-4:177.6_api2_result.json
CPC006_PC3_24H:BRD-K78599730-001-02-6:9.08_api2_result.json
CPC006_PC3_24H:BRD-K78659596-001-01-3:10_api2_result.json
CPC006_PC3_24H:BRD-K78867378-001-07-3:10_api2_result.json
CPC006_PC3_24H:BRD-K78959463-001-05-8:10_api2_result.json
CPC006_PC3_24H:BRD-K79983625-001-01-1:11.1_api2_result.json
CPC006_PC3_24H:BRD-K80738081-001-21-1:10_api2_result.json
CPC006_PC3_24H:BRD-K81418486-001-13-7:11.1_api2_result.j

CPC006_PC3_6H:BRD-K24132293-001-16-0:10_api2_result.json
CPC006_PC3_6H:BRD-K25737009-001-02-0:40_api2_result.json
CPC006_PC3_6H:BRD-K26657438-001-01-2:10_api2_result.json
CPC006_PC3_6H:BRD-K26664453-001-03-4:10_api2_result.json
CPC006_PC3_6H:BRD-K28296557-005-14-6:10_api2_result.json
CPC006_PC3_6H:BRD-K28360340-001-01-8:10_api2_result.json
CPC006_PC3_6H:BRD-K28907958-001-02-7:11.1_api2_result.json
CPC006_PC3_6H:BRD-K30064966-001-01-5:11.1_api2_result.json
CPC006_PC3_6H:BRD-K30677119-001-01-0:11.1_api2_result.json
CPC006_PC3_6H:BRD-K30707190-001-02-5:80_api2_result.json
CPC006_PC3_6H:BRD-K31342827-001-05-4:12.12_api2_result.json
CPC006_PC3_6H:BRD-K33272502-001-01-2:40_api2_result.json
CPC006_PC3_6H:BRD-K33583600-001-09-6:10_api2_result.json
CPC006_PC3_6H:BRD-K34581968-001-01-2:11.1_api2_result.json
CPC006_PC3_6H:BRD-K35716340-001-02-0:12_api2_result.json
CPC006_PC3_6H:BRD-K35960502-001-10-1:10_api2_result.json
CPC006_PC3_6H:BRD-K36927236-001-17-7:10_api2_result.json
CPC006_PC3_6H:BRD-K3

CPC006_PL21_6H:BRD-K01976263-003-04-5:0.63_api2_result.json
CPC006_PL21_6H:BRD-K02526760-001-01-5:10_api2_result.json
CPC006_PL21_6H:BRD-K18190982-002-01-2:44.4_api2_result.json
CPC006_PL21_6H:BRD-K35716340-001-02-0:12_api2_result.json
CPC006_PL21_6H:BRD-K41859756-001-01-9:10_api2_result.json
CPC006_PL21_6H:BRD-K48692744-001-01-2:10_api2_result.json
CPC006_PL21_6H:BRD-K49371609-310-01-1:0.09_api2_result.json
CPC006_PL21_6H:BRD-K52075040-001-03-0:44.4_api2_result.json
CPC006_PL21_6H:BRD-K54233340-300-01-0:10_api2_result.json
CPC006_PL21_6H:BRD-K64890080-001-03-9:10_api2_result.json
CPC006_PL21_6H:BRD-K67566344-001-01-8:10_api2_result.json
CPC006_PL21_6H:BRD-K69840642-001-01-4:10_api2_result.json
CPC006_PL21_6H:BRD-K71726959-001-06-7:122.55_api2_result.json
CPC006_PL21_6H:BRD-K74402642-003-01-2:10_api2_result.json
CPC006_PL21_6H:BRD-K78659596-001-01-3:10_api2_result.json
CPC006_PL21_6H:BRD-K82135108-001-01-9:10_api2_result.json
CPC006_PL21_6H:BRD-K89391146-001-02-3:80_api2_result.json
CP

ConnectionError: HTTPConnectionPool(host='amp.pharm.mssm.edu', port=80): Max retries exceeded with url: /L1000FWD/sig/CPC006_RMGI_6H:BRD-A36630025-001-02-6:0.35 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7faa130d04e0>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [45]:
significant_signatures_names

['AML001_CD34_6H:BRD-K43389675:10',
 'AML001_PC3_6H:BRD-A19037878:0.37037',
 'AML001_PC3_6H:BRD-A19037878:1.11111',
 'AML001_PC3_6H:BRD-A19037878:10',
 'AML001_PC3_6H:BRD-A19037878:3.33333',
 'AML001_PC3_6H:BRD-A19500257:0.37037',
 'AML001_PC3_6H:BRD-A19500257:1.11111',
 'AML001_PC3_6H:BRD-A19500257:10',
 'AML001_PC3_6H:BRD-A19500257:3.33333',
 'AML001_PC3_6H:BRD-A45664787:10',
 'AML001_PC3_6H:BRD-A75409952:0.37037',
 'AML001_PC3_6H:BRD-A75409952:1.11111',
 'AML001_PC3_6H:BRD-A75409952:10',
 'AML001_PC3_6H:BRD-A75409952:3.33333',
 'AML001_PC3_6H:BRD-K27316855:0.37037',
 'AML001_PC3_6H:BRD-K27316855:1.11111',
 'AML001_PC3_6H:BRD-K27316855:10',
 'AML001_PC3_6H:BRD-K27499107:10',
 'AML001_PC3_6H:BRD-K27499107:3.33333',
 'AML001_PC3_6H:BRD-K43389675:0.37037',
 'AML001_PC3_6H:BRD-K43389675:1.11111',
 'AML001_PC3_6H:BRD-K43389675:10',
 'AML001_PC3_6H:BRD-K43389675:3.33333',
 'AML001_PC3_6H:BRD-K93754473:4.48614',
 'AML001_U937_24H:BRD-K43389675:1.11111',
 'AML001_U937_24H:BRD-K43389675:3.333

In [ ]:
list(itertools.chain.from_iterable(L1000_disease_pert_id_store))

In [24]:
### CREEDS DISEASE CARDS
doi_search =EMR_data_df[EMR_data_df['Description'].apply(lambda s: bool(re.compile(str(DOI), re.IGNORECASE).search(str(s))))]
doi_search_top5 = doi_search[0:5]
EMR_top_disease = doi_search_top5["ICD9"]


## build a datatable of all the ICD-9 CM diagnosis codes families (i.e no decimal points)
EMR_top_disease_df = pd.DataFrame(EMR_top_disease, columns=['ICD9'])
EMR_top_disease_df['ICD9_wildcard'] = EMR_top_disease_df['ICD9'].apply(lambda code: code.split('.')[0])
EMR_top_disease_df
icd9_to_doid_final['ICD9_wildcard'] = icd9_to_doid_final['ICD9'].apply(lambda code: str(code).split('.')[0])

disease_df_joined = pd.merge(
    left=EMR_top_disease_df, left_on='ICD9_wildcard',
    right=icd9_to_doid_final, right_on='ICD9_wildcard',
    how='inner',
    suffixes=(
        '_left',
        '_right',
    )
)

disease_df_joined


CREEDS_disease_ids = pd.DataFrame(set(disease_df_joined.CREEDS_drug_id))
CREEDS_disease_ids_list = list(set(disease_df_joined.CREEDS_drug_id))


CREEDS_Disease_Final = dict((k, CREEDS_GSE[k]) for k in CREEDS_disease_ids_list)
CREEDS_Disease_final_df = pd.DataFrame(CREEDS_Disease_Final).T
CREEDS_Disease_final_df.columns = ["GSE_ID", "DISEASE"]
CREEDS_Disease_final_df # this will be the dataframe to display


,GSE_ID,DISEASE
dz:716,GSE41649,allergic asthma
dz:829,GSE43696,asthma
dz:436,GSE3184,allergic asthma
dz:830,GSE43696,asthma
dz:567,GSE27011,asthma
dz:565,GSE27011,asthma
dz:568,GSE27011,asthma
dz:318,GSE4302,asthma
dz:348,GSE2276,allergic asthma
dz:729,GSE16032,asthma


In [55]:
## CREEDS DISEASE CARD FROM DISEASE QUERY 
CREEDS_URL = 'http://amp.pharm.mssm.edu/CREEDS/'
CREEEDS_Disease_response = requests.get(CREEDS_URL + 'search', params={'q':DOI})
if CREEEDS_Disease_response.status_code == 200:
	#pprint(CREEEDS_Disease_response.json())
	#json.dump(CREEEDS_Disease_response.json(), open('api1_result.json', 'wb'), indent=4) # this is the entire json if you want to save it 
    CREEEDS_Disease_response_json = CREEEDS_Disease_response.json() 
CREEEDS_Disease_response_df = pd.DataFrame(CREEEDS_Disease_response_json)
CREEEDS_Disease_response_df # This will be the dataframe to return

CREEDS_dz_id = list(CREEEDS_Disease_response_df.id)

for a in CREEDS_dz_id:
    CREEDS_dz_id_json = requests.get(CREEDS_URL + 'api', params={'id': CREEDS_dz_id[0]})
    CREEDS_sig_dz_json = CREEDS_dz_id_json.json()
    
    ## up genes
    CREEDS_disease_sig_up_genes = CREEDS_sig_dz_json['up_genes']
    CREEDS_disease_sig_up_genes_df = pd.DataFrame(CREEDS_disease_sig_up_genes) # this is the up genes dataframe
    filename1 = (str(a) + "_CREEDS_disease_query_sig_up_genes.csv")
    CREEDS_disease_sig_up_genes_df.to_csv(filename1) # this saves the df as a csv
    
    ## down genes
    CREEDS_disease_sig_down_genes = CREEDS_sig_dz_json['down_genes']
    CREEDS_disease_sig_down_genes_df = pd.DataFrame(CREEDS_disease_sig_down_genes) # this is the up genes dataframe
    filename1 = (str(a) + "_CREEDS_disease_query_sig_down_genes.csv")
    CREEDS_disease_sig_down_genes_df.to_csv(filename1) # this saves the df as a csv


In [53]:
sig

{'do_id': 'DOID:2841',
 'cell_type': 'Epithelial Cell',
 'pert_ids': ['GSM98141',
  'GSM98142',
  'GSM98143',
  'GSM98144',
  'GSM98145',
  'GSM98146',
  'GSM98147',
  'GSM98148',
  'GSM98149',
  'GSM98150',
  'GSM98151',
  'GSM98152',
  'GSM98153',
  'GSM98154',
  'GSM98155',
  'GSM98156',
  'GSM98157',
  'GSM98158',
  'GSM98159',
  'GSM98160',
  'GSM98161',
  'GSM98162',
  'GSM98163',
  'GSM98164',
  'GSM98165',
  'GSM98166',
  'GSM98167',
  'GSM98168',
  'GSM98169',
  'GSM98170',
  'GSM98171',
  'GSM98172',
  'GSM98173',
  'GSM98174',
  'GSM98175',
  'GSM98176',
  'GSM98177',
  'GSM98178',
  'GSM98179',
  'GSM98180',
  'GSM98181',
  'GSM98182',
  'GSM98183',
  'GSM98184',
  'GSM98185',
  'GSM98186',
  'GSM98187',
  'GSM98188',
  'GSM98189',
  'GSM98190',
  'GSM98191',
  'GSM98192',
  'GSM98193',
  'GSM98194',
  'GSM98195',
  'GSM98196',
  'GSM98197',
  'GSM98198',
  'GSM98199',
  'GSM98200',
  'GSM98201',
  'GSM98202',
  'GSM98203',
  'GSM98204',
  'GSM98208',
  'GSM98210',
  'GSM98

In [ ]:
### CREEDS DRUG CARD DISEASE QUERY



#for a in loop_iteration:
CREEDS_URL = 'http://amp.pharm.mssm.edu/CREEDS/'
CREEEDS_Drug_response = requests.get(CREEDS_URL + 'search', params={'q':DrOI})
if CREEEDS_Drug_response.status_code == 200:
    #pprint(CREEEDS_Drug_response.json())
    #json.dump(CREEEDS_Drug_response.json(), open(DrOI + '_api1_result.json', 'w'), indent=4)
    CREEDS_drug_output_df = pd.DataFrame(CREEEDS_Drug_response.json())
    CREEDS_drug_output_ids = list(CREEDS_drug_output_df["id"])
    
    for a in CREEDS_drug_output_ids:
        CREEDS_URL = 'http://amp.pharm.mssm.edu/CREEDS/'
        CREEDS_drug_sigs_response = requests.get(CREEDS_URL + 'api', params={'id':'drug:DM609'})
        if CREEDS_drug_sigs_response.status_code == 200:
            CREEDS_drug_sigs_response_json = CREEDS_drug_sigs_response.json()
            
            ## up genes
            CREEDS_drug_sigs_up_genes = CREEDS_drug_sigs_response_json['up_genes']
            CREEDS_drug_sigs_up_genes_df = pd.DataFrame(CREEDS_drug_sigs_up_genes) # this is the up genes dataframe
            filename1 = (a + "_CREEDS_drug_sig_up_genes.csv")
            CREEDS_drug_sigs_up_genes_df.to_csv(filename1) # this saves the df as a csv
            
            ## down genes
            CREEDS_drug_sigs_down_genes = CREEDS_drug_sigs_response_json['down_genes']
            CREEDS_drug_sigs_down_genes_df = pd.DataFrame(CREEDS_drug_sigs_down_genes)# this is the down genes dataframe
            filename2 = (a + "_CREEDS_drug_sig_down_genes.csv")
            CREEDS_drug_sigs_down_genes_df.to_csv(filename2)
            CREEDS_drug_sigs_down_genes = CREEDS_drug_sigs_response_json['down_genes'] # this saves the df as a csv
            
            ## json propagation
            #pprint(response.json())
            json.dump(response.json(), open(a + '_CREEDS_Drug_sig.json', 'w'), indent=4) # if the user wants the entire json, they can download this
            

In [ ]:
## works but not optimal
test1 = pd.DataFrame()
for a in ICD9_df_joined.ICD9_wildcard.unique():
    test = EMR_data_df[EMR_data_df["ICD9"].str.contains(a)]
    test1 = pd.concat(test1, test)
test

In [83]:
CREEDS_Disease_final_df

,GSE_ID,DISEASE
dz:828,GSE43696,asthma
dz:714,GSE31773,asthma
dz:567,GSE27011,asthma
dz:565,GSE27011,asthma
dz:827,GSE43696,asthma
dz:716,GSE41649,allergic asthma
dz:348,GSE2276,allergic asthma
dz:318,GSE4302,asthma
dz:360,GSE3004,allergic asthma
dz:634,GSE43696,asthma
